In [ ]:
pip install qrisp

In [1]:
from qrisp import *
from qrisp.interface import VirtualQiskitBackend
from qiskit_alice_bob_provider.local.provider import AliceBobLocalProvider

provider = AliceBobLocalProvider()
backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')
vrtl_qasm_sim = VirtualQiskitBackend(backend)

In [2]:
N = 15

a = 2

qg = QuantumModulus(N)

qg[:] = 1

In [3]:
n = qg.size

qpe_res = QuantumFloat(2*n+1, exponent = -(2*n+1))

h(qpe_res)

<QuantumFloat 'qpe_res'>

In [4]:
x = a
for i in range(len(qpe_res)):
    with control(qpe_res[i]):
        qg *= x
        x = (x*x)%N

In [5]:
QFT(qpe_res, inv = True)

# print(qpe_res.qs)

<QuantumFloat 'qpe_res'>

In [6]:
meas_res = qpe_res.get_measurement(backend=vrtl_qasm_sim)
#meas_res = qpe_res.get_measurement()
#print(meas_res)

#qasm_str = qpe_res.qs.compile().qasm()

#print(qasm_str)

#from qiskit import QuantumCircuit

#circ = QuantumCircuit.from_qasm_str(qasm_str)

#from qiskit import QuantumCircuit
#from qiskit_alice_bob_provider.local.provider import AliceBobLocalProvider
#from qiskit import transpile, execute

#print(circ)

#provider = AliceBobLocalProvider()
#backend = provider.get_backend('EMU:40Q:LOGICAL_TARGET')

#circ = transpile(circ, backend)

ERROR:stevedore.extension:Could not load 'ibm_backend': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_backend': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_backend': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_backend': No module named 'qiskit.circuit.singleton'
ERROR:stevedore.extension:Could not load 'ibm_dynamic_circuits': No module named 'qiskit.circuit.singleton'


In [7]:
from sympy import continued_fraction_convergents, continued_fraction_iterator, Rational

def get_r_candidates(approx):
    rationals = continued_fraction_convergents(continued_fraction_iterator(Rational(approx)))
    return [rat.q for rat in rationals]

In [8]:
r_candidates = sum([get_r_candidates(approx) for approx in meas_res.keys()], [])

In [9]:
for cand in r_candidates:
    if (a**cand)%N == 1:
        r = cand
        break
else:
    raise Exception("Please sample again")

if r % 2:
    raise Exception("Please choose another a")

In [10]:
import numpy as np

g = np.gcd(a**(r//2)+1, N)

print(g)

5


In [11]:
def find_order(a, N):
    qg = QuantumModulus(N)
    qg[:] = 1
    qpe_res = QuantumFloat(2*qg.size + 1, exponent = -(2*qg.size + 1))
    h(qpe_res)
    for i in range(len(qpe_res)):
        with control(qpe_res[i]):
            qg *= a
            a = (a*a)%N
    QFT(qpe_res, inv = True)
    return qpe_res.get_measurement()